# Run Vcontact2 with the prophage genomes
***

Generate the input files 

In [ ]:
# Create a tmp with all the candidates :

import os
import pandas as pd
import random
import sys
import subprocess
from tqdm import tqdm


path_phageboost_pred="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_prediction"
path_fasta="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
path_fastani = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_ktype="/home/conchae/prediction_depolymerase_tropism"


df_representant = pd.read_csv(f"{path_fastani}/clusters_99_80.extra_clean.2004.v2.tsv", sep = "\t", header = 0)
df_representant = df_representant.drop_duplicates(subset = ["prophage_id"], keep = "first")

dico_prophage = {}

for _,row in df_representant.iterrows() :
    strain = row["prophage"].split("__")[0]
    prophage = row["prophage"].split(".fasta")[0].split("__")[-1]
    seq_id = row["prophage_id"]
    a = {"strain" : strain, "prophage" : prophage, "prophage_n" : seq_id}
    dico_prophage[row["prophage"]] = a
        
for phage in tqdm(dico_prophage):       
    strain = dico_prophage[phage]["strain"]
    prophage_id = dico_prophage[phage]["prophage"]
    prophage = phage.split(".fasta")[0]
    genecall = pd.read_csv(f"{path_phageboost_pred}/{strain}/genecalls_{strain.split('.')[0]}.gff3", sep="\t")
    for file in os.listdir(f"{path_phageboost_pred}/{strain}"):
        if file[0:6]=="phages":
            resume= pd.read_csv(f"{path_phageboost_pred}/{strain}/{file}", skiprows=1, sep="\t")
            break
    attributes = resume["attributes"].to_list()
    for index_att, attri in enumerate(attributes):
        if attri.split("phage_id=")[1] == prophage_id :
            attribute_line = attri
            break
    frag_id = resume[resume["attributes"] == attribute_line]["#seqid"].values[0]
    start_genome ,stop_genome =resume[resume["attributes"] == attribute_line]["start"].values[0], resume[resume["attributes"] == attribute_line]["end"].values[0]
    print(frag_id ,start_genome, stop_genome)
    #break
    genecall_frag = genecall[genecall["contig"] == frag_id]
    genecall_frag_dict = genecall_frag.to_dict('records')
    with open(f"{path_decipher}/{strain}/{prophage_id}.multi.faa", "w") as outfile_faa :
        with open(f"{path_decipher}/{strain}/{prophage_id}.multi.ffn", "w") as outfile_ffn :
            for line in genecall_frag_dict : 
                if line["start"] in range(start_genome, stop_genome) :
                    nt_seq, aa_seq, prot_id =line["DNAseq"] , line["AAseq"],  line["id"]
                    print(len(aa_seq), "Protein_id : ", prot_id)
                    if len(aa_seq) > 10 :
                        outfile_faa.write(f">{strain}__{prophage_id}__{prot_id}\n{aa_seq}\n")
                        outfile_ffn.write(f">{strain}__{prophage_id}__{prot_id}\n{nt_seq}\n")
                        print(f">{strain}__{prophage_id}__{prot_id}\n{aa_seq}\n", f">{strain}__{prophage_id}__{prot_id}\n{nt_seq}\n")
                
    

In [ ]:
#!/bin/bash
#BATCH --job-name=write_multi
#SBATCH --qos=short 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=4
#SBATCH --mem=5gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=write_multi%j.log 

module restore la_base
conda activate python_311
                
python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/script_file/make_multifasta.py

In [ ]:
# Step one : generating the fasta file with the phages proteins and the gene_to_genome file
import os
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO

path_phageboost = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_phylogeny = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny"
path_fastani = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"

df_representant = pd.read_csv(f"{path_fastani}/clusters_99_80.extra_clean.2004.v2.tsv", sep = "\t", header = 0)
df_representant = df_representant.drop_duplicates(subset = ["prophage_id"], keep = "first")

dico_prophage = {}

for _,row in df_representant.iterrows() :
    strain = row["prophage"].split("__")[0]
    prophage = row["prophage"].split(".fasta")[0].split("__")[-1]
    seq_id = row["prophage_id"]
    a = {"strain" : strain, "prophage" : prophage, "prophage_n" : seq_id}
    dico_prophage[row["prophage"]] = a
    
with open(f"{path_phylogeny}/phageboost_70_Vcontact2.0611.fasta","w") as outfile1 :
    with open(f"{path_phylogeny}/phageboost_70_gene_to_genome.0611.csv","w") as outfile2:
        outfile2.write("protein_id,contig_id,keywords\n")
        for prophage in tqdm(dico_prophage) :
            path_multi = f"{path_phageboost}/{dico_prophage[prophage]['strain']}/{dico_prophage[prophage]['prophage']}.multi.faa"
            fasta_file = SeqIO.parse(path_multi , "fasta")
            for index, record in enumerate(fasta_file) :
                prot_id = f"{prophage.split('.fasta')[0]}__{index}"
                outfile1.write(f">{prot_id}\n{str(record.seq)}\n")
                outfile2.write(f"{prot_id},{prophage.split('.fasta')[0]},None_provided\n")
                


Merge the previously created input files with the Millard data

In [ ]:
import os

path_phageboost="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70"
path_phylogeny="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny"
path_77="/home/conchae/77_strains_phage_project"
path_millard = "/home/conchae/databases/Millard_jan_2023"

gene2genome_m = "\n".join(open(f"{path_millard}/5Jan2023_vConTACT2_gene_to_genome.csv").read().split("\n")[1:])
gene_seq_m = open(f"{path_millard}/5Jan2023_vConTACT2_proteins.faa").read()

PPT_gene2genome = open(f"{path_phylogeny}/phageboost_70_gene_to_genome.0611.csv").read()
PPT_gene_seq = open(f"{path_phylogeny}/phageboost_70_Vcontact2.0611.fasta").read()

with open(f"{path_phylogeny}/phageboost_70_Vcontact2.Millard.0611.fasta","w") as outfile :
    with open(f"{path_phylogeny}/phageboost_70_gene_to_genome.Millard.0611.csv","w") as outfile2:
        outfile.write(f"{PPT_gene_seq}\n{gene_seq_m}")
        outfile2.write(f"{PPT_gene2genome}\n{gene2genome_m}")
        
        

In [ ]:
# Step two : The actual command line :

#!/bin/bash
#BATCH --job-name=vcontact2_phageboost
#SBATCH --qos=medium 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=40
#SBATCH --mem=200gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=vcontact2_phageboost%j.log 

module restore la_base
source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate vcontact2


vcontact --rel-mode 'Diamond' \
--raw-proteins /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_Vcontact2.Millard.0611.fasta \
--proteins-fp /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_gene_to_genome.Millard.0611.csv \
--output-dir  /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023 \
--db 'None'  --pcs-mode MCL --vcs-mode ClusterONE --c1-bin /home/conchae/software/cluster_one-1.0.jar -t 40



In [ ]:
from os import listdir, mkdir, rename, system
import os

path_77="/home/conchae/77_strains_phage_project"
path_77_seq="/home/conchae/77_strains_phage_project/phages_sequences_corrected"

vcontact2_results=open(path_77+"/vConTACT2_77_phages_V5/genome_by_genome_overview.csv").read().split("\n")
reference={}
our_phages=[]

for phage in os.listdir(path_77_seq):
    our_phages.append(phage)
    
for n in range (1,len(vcontact2_results)-1):
    phage=vcontact2_results[n].split(",")[1]
    if phage not in our_phages:
        V_C=vcontact2_results[n].split(",")[5]
        order=vcontact2_results[n].split(",")[2]
        family=vcontact2_results[n].split(",")[3]
        genus=vcontact2_results[n].split(",")[4]
        score=vcontact2_results[n].split(",")[-1]
        if V_C !="" and V_C not in reference and float(score) > 0.9 :
            a={"Order":order}
            b={"Family":family}
            c={"Genus":genus}
            a.update(b)
            a.update(c)
            reference[V_C]=a

with open(path_77+"/Vcontact_77_results.tsv","w") as outfile:
    for n in range(1,len(vcontact2_results)-1):
        phage=vcontact2_results[n].split(",")[1]
        if phage in our_phages:
            V_C=vcontact2_results[n].split(",")[5]
            score=vcontact2_results[n].split(",")[-1]
            try :
                if float(score) > 0.9 and V_C in reference:
                    outfile.write(phage+"\t"+reference[V_C]["Order"]+"\t"+reference[V_C]["Family"]+"\t"+reference[V_C]["Genus"]+"\t"+V_C+"\t"+score+"\n")
                elif float(score) > 0.7 and V_C in reference :
                    outfile.write(phage+"\t"+reference[V_C]["Order"]+"\t"+reference[V_C]["Family"]+"\t"+V_C+"\t"+score+"\n")
                elif V_C =="67_0":
                    outfile.write(phage+"\t Caudovirales \t Siphoviridae \n")
                elif V_C not in reference :
                    print(phage, V_C)
                    outfile.write(vcontact2_results[n]+"\n")
                else:
                    print(phage)
            except ValueError:
                outfile.write(vcontact2_results[n]+"\n")